# Example for:
Transfer Learning Empirical Experiment from Vehicles (pretraining) to Animals (finetuning) classes of CIFAR

### Setup and Hyperparams

In [7]:
# Specify which gpu
import os
gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

import sys
sys.path.append('/home/arnisaf/mp-tl-study')
from functions.utils import *

# device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)  # if using multi-GPU

In [8]:
# cuts=0 means: end-to-end model if we are reinitializing
cuts = [0,1,2,3,4,5,6]

In [9]:
# Changes Here for the experiments
params = {
      # MODEL ARCHITECTURE PARAMS
      'depth': 6,
      'num_channels': 64,
      'width':64,
      # 'hidden_dim_lin': 128,
      'activation_function': nn.ReLU,
      'kernel_size': 3,
      # TRAINING PARAMS
      'device': device,
      'lr_pretrain': 0.001,   
      'lr_fine_tune': 0.001,  # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_reinit': 0.001,         # CHANGE: if no layer-wise lr
      # 'lr_fine_tune_no_reinit': 0.0001,     # CHANGE: if layer-wise lr
      'num_train': 10,
      'early_stop_patience': 5,
      'save_best': False,
      'save_checkpoints': False,
      'is_cnn': True,
      'is_debug': False,
      'classification_report_flag': False,
      'batch_size':64,
      # DATASET PARAMS
      'pre_train_classes': [2, 3, 4, 5, 6, 7],
      'fine_tune_classes': [0, 1, 8, 9],
      'val_split': 0.1,
      'num_workers': 0,
      'generate_dataset_seed': 42,
      # EXPERIMENT SETTING PARAMS
      'use_pooling': True,   # CHANGE
      'pooling_every_n_layers': 2, # add pooling after every n layers specified here. For only one pooling after all the CNN layers, this equals params['depth']
      'pooling_stride': 2,
      'freeze': True,         # CHANGE: freeze the conv layers before the cut
      'reinit': True,         # CHANGE: reinit the conv lyers only after the cut
      'reinit_both_dense': True,   # CHANGE: True for reinitialize both dense layers, False for reinit only the last dense layer
      'truncate': False
    }

In [4]:
root_dir = './data'  # Specify your data directory here
transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
dataloader_wrapped = TransferLearningWrapper(params, datasets.CIFAR10, datasets.CIFAR10, root_dir, transform=transform)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
dataloader_wrapped.pretrain_train_loader.dataset.__len__()

27000

## Pretraining

In [6]:
pretrained_model = CustomCNN(params, dataloader_wrapped.output_dim, tuple(dataloader_wrapped.finetune_test_loader.dataset[0][0].shape))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=6, bias=True)
)

In [7]:
# Train and evaluate - Skip if loading saved model!
trainer = Trainer(pretrained_model, dataloader_wrapped, params["lr_pretrain"], params)
train_acc, test_acc, effective_epochs, checkpoints = trainer.train()

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Test Accuracy: {test_acc:.4f}")

Epoch: 0 	Training Accuracy: 46.32%
Validation Accuracy: 44.77%
Epoch: 1 	Training Accuracy: 55.19%
Validation Accuracy: 53.07%
Epoch: 2 	Training Accuracy: 62.42%
Validation Accuracy: 59.77%
Epoch: 3 	Training Accuracy: 67.20%
Validation Accuracy: 63.07%
Epoch: 4 	Training Accuracy: 70.06%
Validation Accuracy: 65.37%
Epoch: 5 	Training Accuracy: 73.70%
Validation Accuracy: 68.43%
Epoch: 6 	Training Accuracy: 75.85%
Validation Accuracy: 69.23%
Epoch: 7 	Training Accuracy: 77.35%
Validation Accuracy: 70.67%
Epoch: 8 	Training Accuracy: 78.44%
Validation Accuracy: 70.43%
Epoch: 9 	Training Accuracy: 81.73%
Validation Accuracy: 72.67%
Final Training Accuracy: 0.8173
Final Test Accuracy: 0.8083


In [9]:
torch.save(pretrained_model.state_dict(), 'pretrained_models/pretrained_model_animals/pretrained_model_animals_new.pth')

In [7]:
pretrained_model.load_state_dict(torch.load('pretrained_models/pretrained_model_animals/pretrained_model_animals_new.pth'))
pretrained_model.to(device)

CustomCNN(
  (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act0): ReLU()
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act1): ReLU()
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act2): ReLU()
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act3): ReLU()
  (pool3): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act4): ReLU()
  (conv5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (act5): ReLU()
  (pool5): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc): Linear(in_features=1024, out_features=6, bias=True)
)

In [8]:
eval(pretrained_model, device, dataloader_wrapped.test_loader, debug=True, classification_report_flag=True, is_cnn=True)


Average loss: 0.5366, Accuracy: 24248.0/30000 (81%)

              precision    recall  f1-score   support

     Class 0       0.84      0.80      0.82      5000
     Class 1       0.65      0.80      0.72      5000
     Class 2       0.85      0.80      0.82      5000
     Class 3       0.82      0.70      0.76      5000
     Class 4       0.82      0.93      0.87      5000
     Class 5       0.94      0.83      0.88      5000

    accuracy                           0.81     30000
   macro avg       0.82      0.81      0.81     30000
weighted avg       0.82      0.81      0.81     30000



0.8082666666666667

## Fine-tuning Experiments

### Baselines (End-to-end models trained on subsets of fine-tuning dataset)
We also reuse the baselines a lot! so skip if we already have the jsons

In [12]:
baselines_results = []
percentages = [0.001, 0.01, 0.1, 0.5, 1.0]

In [13]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:      
    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
    
    for repeat in range(repeats):
        # Print or log the sampled values for transparency
        print(f"\nSampled Percentage: {sampled_percentage}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

        # Reduce the dataset
        train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        torch.manual_seed(repeat)
        #train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed = repeat)
        dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)

        # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
        params_tmp = copy.deepcopy(params)
        params_tmp["reinit"] = True
        model_new = cut_custom_cnn_model(pretrained_model, cut_point=0, params=params_tmp, output_dim=dataloader_wrapped.output_dim)
        model_new.to(device)

        # Train and evaluate
        trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
        train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
        print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

        # Store the results
        baselines_results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":-1, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc}) # -1 for the cut point means it's baseline


Sampled Percentage: 0.001, Lr: 0.001, Repeat: 0
Early stopping invoked.
Training Accuracy: 0.3750, Test Accuracy: 0.2961

Sampled Percentage: 0.001, Lr: 0.001, Repeat: 1


In [ ]:
print(baselines_results)

In [15]:
# save baseline results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + baselines_results

with open(f'results/cifarstyle_baselines_cifar_reversed.json', 'w') as f:
    json.dump(results, f)

### Fine-tuning

In [9]:
results = []
#percentages = [0.001, 0.01, 0.1, 0.5, 1.0]
percentages = [1.0]

In [10]:
dataloader_wrapped.update_phase('finetune')

for sampled_percentage in percentages:

    if sampled_percentage <= 0.01:
        repeats = 25
    elif sampled_percentage < 0.5:
        repeats = 20
    else:
        repeats = 5
        
    for sampled_cut_point in cuts:

        for repeat in range(repeats):
            # Add the combination to the tested set
            # tested_combinations.add((sampled_percentage, sampled_cut_point))

            # Print or log the sampled values for transparency
            print(f"\nSampled Percentage: {sampled_percentage}, Sampled Cut Point: {sampled_cut_point}, Lr: {params['lr_fine_tune']}, Repeat: {repeat}")

            # Reduce the dataset
            train_loader_reduced = reduce_dataset(dataloader_wrapped.train_loader, sampled_percentage, seed=repeat)
            dataset_namespace_new = SimpleNamespace(train_loader=train_loader_reduced, test_loader=dataloader_wrapped.test_loader, val_loader=dataloader_wrapped.val_loader)
            torch.manual_seed(repeat) # because in the cut function we reinitialize some layers too (at least the dense layers)
            
            # Copy and then cut the model - we already deepcopy it in the function: pretrained_model
            model_new = cut_custom_cnn_model(pretrained_model, cut_point=sampled_cut_point, params=params, output_dim=dataloader_wrapped.output_dim)
            model_new.to(device)
            
            # Train and evaluate
            trainer = Trainer(model_new, dataset_namespace_new, params['lr_fine_tune'], params)
            train_acc, test_acc, effective_epochs, checkpoints = trainer.train(verbose=0)
            print(f"Training Accuracy: {train_acc:.4f}, Test Accuracy: {test_acc:.4f}")

            # Store the results
            results.append({"lr":params['lr_fine_tune'], "sampled_percentage":sampled_percentage, "sampled_cut_point":sampled_cut_point, "repeat":repeat, "train_acc":train_acc, "test_acc":test_acc})


Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9508, Test Accuracy: 0.9441

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9483, Test Accuracy: 0.9418

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9488, Test Accuracy: 0.9431

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 3
Training Accuracy: 0.9488, Test Accuracy: 0.9428

Sampled Percentage: 1.0, Sampled Cut Point: 0, Lr: 0.001, Repeat: 4
Training Accuracy: 0.9573, Test Accuracy: 0.9505

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 0
Training Accuracy: 0.9389, Test Accuracy: 0.9338

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 1
Training Accuracy: 0.9473, Test Accuracy: 0.9411

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.001, Repeat: 2
Training Accuracy: 0.9478, Test Accuracy: 0.9419

Sampled Percentage: 1.0, Sampled Cut Point: 1, Lr: 0.00

In [11]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.950824237023836, 'test_acc': 0.9441}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9482624192470483, 'test_acc': 0.94185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9488193361550457, 'test_acc': 0.9431}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9488193361550457, 'test_acc': 0.9428}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9573401648474048, 'test_acc': 0.95045}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9389062151926932, 'test_acc': 0.93385}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9473156605034528, 'test_acc': 0.94105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat

In [12]:
print(results)

[{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.950824237023836, 'test_acc': 0.9441}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9482624192470483, 'test_acc': 0.94185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9488193361550457, 'test_acc': 0.9431}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9488193361550457, 'test_acc': 0.9428}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9573401648474048, 'test_acc': 0.95045}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9389062151926932, 'test_acc': 0.93385}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9473156605034528, 'test_acc': 0.94105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat

In [5]:
baselines_results = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.375, 'test_acc': 0.2961}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.4375, 'test_acc': 0.3585}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.75, 'test_acc': 0.3408}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.75, 'test_acc': 0.31205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.8125, 'test_acc': 0.2421}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.6875, 'test_acc': 0.2935}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.6875, 'test_acc': 0.35795}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.8125, 'test_acc': 0.3997}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.75, 'test_acc': 0.33875}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.3125, 'test_acc': 0.2498}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.5, 'test_acc': 0.3524}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.75, 'test_acc': 0.39555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.625, 'test_acc': 0.33}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.6875, 'test_acc': 0.3027}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.625, 'test_acc': 0.3735}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.6875, 'test_acc': 0.3647}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.3125, 'test_acc': 0.2599}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.625, 'test_acc': 0.3569}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.4375, 'test_acc': 0.29735}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.6875, 'test_acc': 0.35565}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.6875, 'test_acc': 0.3248}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.5625, 'test_acc': 0.38205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.6875, 'test_acc': 0.37995}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.625, 'test_acc': 0.2877}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.625, 'test_acc': 0.2782}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.49444444444444446, 'test_acc': 0.431}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.5777777777777777, 'test_acc': 0.46035}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.55, 'test_acc': 0.4657}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.5222222222222223, 'test_acc': 0.41995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.4444444444444444, 'test_acc': 0.42995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.5222222222222223, 'test_acc': 0.4342}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.5277777777777778, 'test_acc': 0.4399}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.5833333333333334, 'test_acc': 0.47085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.5111111111111111, 'test_acc': 0.4637}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.4666666666666667, 'test_acc': 0.43625}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.45555555555555555, 'test_acc': 0.44475}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.45555555555555555, 'test_acc': 0.4471}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.5055555555555555, 'test_acc': 0.4514}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.5, 'test_acc': 0.48085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.5555555555555556, 'test_acc': 0.456}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.5666666666666667, 'test_acc': 0.44745}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.5555555555555556, 'test_acc': 0.46035}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.55, 'test_acc': 0.46135}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.49444444444444446, 'test_acc': 0.4422}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.55, 'test_acc': 0.45605}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 20, 'train_acc': 0.5111111111111111, 'test_acc': 0.45355}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 21, 'train_acc': 0.5777777777777777, 'test_acc': 0.44915}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 22, 'train_acc': 0.5611111111111111, 'test_acc': 0.47945}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 23, 'train_acc': 0.46111111111111114, 'test_acc': 0.45595}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': -1, 'repeat': 24, 'train_acc': 0.5333333333333333, 'test_acc': 0.46675}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.6955555555555556, 'test_acc': 0.6504}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.7155555555555555, 'test_acc': 0.63905}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.7144444444444444, 'test_acc': 0.6419}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.6983333333333334, 'test_acc': 0.64965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.7027777777777777, 'test_acc': 0.63285}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 5, 'train_acc': 0.7105555555555556, 'test_acc': 0.6449}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 6, 'train_acc': 0.6955555555555556, 'test_acc': 0.63585}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 7, 'train_acc': 0.7022222222222222, 'test_acc': 0.6487}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 8, 'train_acc': 0.6933333333333334, 'test_acc': 0.6459}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 9, 'train_acc': 0.6861111111111111, 'test_acc': 0.6249}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 10, 'train_acc': 0.6955555555555556, 'test_acc': 0.6449}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 11, 'train_acc': 0.6855555555555556, 'test_acc': 0.6222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 12, 'train_acc': 0.7166666666666667, 'test_acc': 0.6432}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 13, 'train_acc': 0.7305555555555555, 'test_acc': 0.6614}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 14, 'train_acc': 0.6588888888888889, 'test_acc': 0.62425}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 15, 'train_acc': 0.6822222222222222, 'test_acc': 0.63365}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 16, 'train_acc': 0.6983333333333334, 'test_acc': 0.63965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 17, 'train_acc': 0.705, 'test_acc': 0.6434}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 18, 'train_acc': 0.6944444444444444, 'test_acc': 0.6118}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': -1, 'repeat': 19, 'train_acc': 0.7466666666666667, 'test_acc': 0.66395}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.9225555555555556, 'test_acc': 0.88365}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9215555555555556, 'test_acc': 0.8831}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9128888888888889, 'test_acc': 0.8708}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.903, 'test_acc': 0.8696}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.912, 'test_acc': 0.87415}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 0, 'train_acc': 0.950824237023836, 'test_acc': 0.9441}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 1, 'train_acc': 0.9482624192470483, 'test_acc': 0.94185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 2, 'train_acc': 0.9488193361550457, 'test_acc': 0.9431}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 3, 'train_acc': 0.9488193361550457, 'test_acc': 0.9428}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': -1, 'repeat': 4, 'train_acc': 0.9573401648474048, 'test_acc': 0.95045}]
finetune1 = [{'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.375, 'test_acc': 0.2961}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.4375, 'test_acc': 0.3585}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.75, 'test_acc': 0.3408}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.75, 'test_acc': 0.31205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.8125, 'test_acc': 0.2421}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.6875, 'test_acc': 0.2935}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.6875, 'test_acc': 0.35795}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.8125, 'test_acc': 0.3997}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.75, 'test_acc': 0.33875}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.3125, 'test_acc': 0.2498}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.5, 'test_acc': 0.3524}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.75, 'test_acc': 0.39555}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.625, 'test_acc': 0.33}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.6875, 'test_acc': 0.3027}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.625, 'test_acc': 0.3735}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.6875, 'test_acc': 0.3647}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.3125, 'test_acc': 0.2599}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.625, 'test_acc': 0.3569}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.4375, 'test_acc': 0.29735}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.6875, 'test_acc': 0.35565}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.6875, 'test_acc': 0.3248}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.5625, 'test_acc': 0.38205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.6875, 'test_acc': 0.37995}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.625, 'test_acc': 0.2877}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.625, 'test_acc': 0.2782}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.75, 'test_acc': 0.36335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.8125, 'test_acc': 0.36195}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.4375, 'test_acc': 0.3093}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.6875, 'test_acc': 0.33045}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.625, 'test_acc': 0.3394}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.875, 'test_acc': 0.36545}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.4375, 'test_acc': 0.28135}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.5, 'test_acc': 0.31045}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.6875, 'test_acc': 0.37335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.625, 'test_acc': 0.31185}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.3125, 'test_acc': 0.3073}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.75, 'test_acc': 0.33505}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.8125, 'test_acc': 0.3155}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.5625, 'test_acc': 0.3728}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.6875, 'test_acc': 0.36905}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.625, 'test_acc': 0.36235}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.3125, 'test_acc': 0.25485}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.5625, 'test_acc': 0.36605}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.5, 'test_acc': 0.31615}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.8125, 'test_acc': 0.3626}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.5, 'test_acc': 0.2919}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.375, 'test_acc': 0.31285}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.5, 'test_acc': 0.2579}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.8125, 'test_acc': 0.3807}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.625, 'test_acc': 0.36025}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.875, 'test_acc': 0.41085}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.4126}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.4375, 'test_acc': 0.33205}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.875, 'test_acc': 0.3189}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.6875, 'test_acc': 0.3594}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.8125, 'test_acc': 0.3797}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.875, 'test_acc': 0.40145}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.8125, 'test_acc': 0.41845}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.4375, 'test_acc': 0.26345}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.3306}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.8125, 'test_acc': 0.343}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.9375, 'test_acc': 0.4059}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.9375, 'test_acc': 0.39895}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.9375, 'test_acc': 0.3168}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.75, 'test_acc': 0.40135}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.6875, 'test_acc': 0.385}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.5625, 'test_acc': 0.31915}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.625, 'test_acc': 0.3319}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.9375, 'test_acc': 0.4039}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.4375, 'test_acc': 0.27895}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.75, 'test_acc': 0.3481}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.625, 'test_acc': 0.35565}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.75, 'test_acc': 0.3124}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.5625, 'test_acc': 0.28805}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.46115}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.5625, 'test_acc': 0.35265}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.75, 'test_acc': 0.38305}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9375, 'test_acc': 0.40105}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.5, 'test_acc': 0.37265}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.875, 'test_acc': 0.38765}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.875, 'test_acc': 0.425}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.9375, 'test_acc': 0.37795}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.8125, 'test_acc': 0.39425}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.9375, 'test_acc': 0.44455}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.35165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.9375, 'test_acc': 0.4204}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.4288}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.9375, 'test_acc': 0.43585}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.38705}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.875, 'test_acc': 0.454}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.4284}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.875, 'test_acc': 0.3242}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.9375, 'test_acc': 0.4497}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.875, 'test_acc': 0.38775}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.8125, 'test_acc': 0.42355}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.4581}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.9375, 'test_acc': 0.40935}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.9375, 'test_acc': 0.40315}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.9375, 'test_acc': 0.41015}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.5625, 'test_acc': 0.2836}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.47345}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.4403}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9375, 'test_acc': 0.44875}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9375, 'test_acc': 0.42225}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.4711}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.46175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.41705}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.43035}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.47905}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.45445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.45865}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.45715}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.4408}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.4186}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.48335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.43445}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.45345}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.9375, 'test_acc': 0.459}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.4401}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.9375, 'test_acc': 0.4122}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.47655}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.4025}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.40005}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.9375, 'test_acc': 0.43175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.46375}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.5054}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.43005}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.48575}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.44515}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.459}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.4921}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.43175}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.465}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.49745}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.45885}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.43845}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.48165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.4201}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.44}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.51615}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.438}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.4591}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.48755}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.43845}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.4243}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.4474}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.44165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.40525}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 1.0, 'test_acc': 0.4876}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.489}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 1.0, 'test_acc': 0.43995}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 1.0, 'test_acc': 0.4397}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 1.0, 'test_acc': 0.4651}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 1.0, 'test_acc': 0.36745}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 1.0, 'test_acc': 0.45165}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 1.0, 'test_acc': 0.39945}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 1.0, 'test_acc': 0.4012}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 1.0, 'test_acc': 0.41595}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 1.0, 'test_acc': 0.4446}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 1.0, 'test_acc': 0.43975}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 1.0, 'test_acc': 0.4418}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 1.0, 'test_acc': 0.43345}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 1.0, 'test_acc': 0.43335}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 1.0, 'test_acc': 0.4287}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 1.0, 'test_acc': 0.3938}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 1.0, 'test_acc': 0.3558}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 1.0, 'test_acc': 0.4064}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 1.0, 'test_acc': 0.4653}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 1.0, 'test_acc': 0.41525}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 1.0, 'test_acc': 0.4226}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 1.0, 'test_acc': 0.3726}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 1.0, 'test_acc': 0.4317}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 1.0, 'test_acc': 0.4148}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.9375, 'test_acc': 0.41635}, {'lr': 0.001, 'sampled_percentage': 0.001, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 1.0, 'test_acc': 0.3964}]
finetune2 = [{'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.49444444444444446, 'test_acc': 0.431}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.5777777777777777, 'test_acc': 0.46035}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.55, 'test_acc': 0.4657}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.5222222222222223, 'test_acc': 0.41995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.4444444444444444, 'test_acc': 0.42995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.5222222222222223, 'test_acc': 0.4342}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.5277777777777778, 'test_acc': 0.4399}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.5833333333333334, 'test_acc': 0.47085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.5111111111111111, 'test_acc': 0.4637}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.4666666666666667, 'test_acc': 0.43625}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.45555555555555555, 'test_acc': 0.44475}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.45555555555555555, 'test_acc': 0.4471}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.5055555555555555, 'test_acc': 0.4514}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.5, 'test_acc': 0.48085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.5555555555555556, 'test_acc': 0.456}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.5666666666666667, 'test_acc': 0.44745}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.5555555555555556, 'test_acc': 0.46035}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.55, 'test_acc': 0.46135}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.49444444444444446, 'test_acc': 0.4422}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.55, 'test_acc': 0.45605}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 20, 'train_acc': 0.5111111111111111, 'test_acc': 0.45355}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 21, 'train_acc': 0.5777777777777777, 'test_acc': 0.44915}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 22, 'train_acc': 0.5611111111111111, 'test_acc': 0.47945}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 23, 'train_acc': 0.46111111111111114, 'test_acc': 0.45595}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 0, 'repeat': 24, 'train_acc': 0.5333333333333333, 'test_acc': 0.46675}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.5055555555555555, 'test_acc': 0.4613}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.5944444444444444, 'test_acc': 0.4812}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.5055555555555555, 'test_acc': 0.4925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.5666666666666667, 'test_acc': 0.46625}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.4888888888888889, 'test_acc': 0.4654}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.4166666666666667, 'test_acc': 0.3936}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.5444444444444444, 'test_acc': 0.4958}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.5555555555555556, 'test_acc': 0.47685}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.5166666666666667, 'test_acc': 0.47635}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.5388888888888889, 'test_acc': 0.4334}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.3388888888888889, 'test_acc': 0.32555}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.5722222222222222, 'test_acc': 0.45795}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.5555555555555556, 'test_acc': 0.44915}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.5166666666666667, 'test_acc': 0.4623}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.5055555555555555, 'test_acc': 0.44065}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.5833333333333334, 'test_acc': 0.4659}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.5, 'test_acc': 0.43255}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.48333333333333334, 'test_acc': 0.46185}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.4888888888888889, 'test_acc': 0.42505}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.5888888888888889, 'test_acc': 0.453}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 20, 'train_acc': 0.5388888888888889, 'test_acc': 0.459}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 21, 'train_acc': 0.5555555555555556, 'test_acc': 0.46145}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 22, 'train_acc': 0.5444444444444444, 'test_acc': 0.472}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 23, 'train_acc': 0.5666666666666667, 'test_acc': 0.4711}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 1, 'repeat': 24, 'train_acc': 0.5444444444444444, 'test_acc': 0.46805}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.5833333333333334, 'test_acc': 0.4843}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.65, 'test_acc': 0.5213}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.5333333333333333, 'test_acc': 0.4658}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.6888888888888889, 'test_acc': 0.5295}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.5888888888888889, 'test_acc': 0.4868}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.5888888888888889, 'test_acc': 0.4885}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.65, 'test_acc': 0.5318}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.6166666666666667, 'test_acc': 0.5349}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.5944444444444444, 'test_acc': 0.44435}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.5611111111111111, 'test_acc': 0.4894}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.7166666666666667, 'test_acc': 0.5172}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.6388888888888888, 'test_acc': 0.5211}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.5222222222222223, 'test_acc': 0.45155}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.6055555555555555, 'test_acc': 0.5261}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.5888888888888889, 'test_acc': 0.51565}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.5111111111111111, 'test_acc': 0.47715}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.6111111111111112, 'test_acc': 0.5303}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.6055555555555555, 'test_acc': 0.4997}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.6333333333333333, 'test_acc': 0.47835}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.5888888888888889, 'test_acc': 0.462}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 20, 'train_acc': 0.5833333333333334, 'test_acc': 0.47745}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 21, 'train_acc': 0.6111111111111112, 'test_acc': 0.4939}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 22, 'train_acc': 0.5777777777777777, 'test_acc': 0.4939}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 23, 'train_acc': 0.6, 'test_acc': 0.47795}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 2, 'repeat': 24, 'train_acc': 0.6444444444444445, 'test_acc': 0.4916}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.7, 'test_acc': 0.545}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.6666666666666666, 'test_acc': 0.52335}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.65, 'test_acc': 0.55045}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.7277777777777777, 'test_acc': 0.5385}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.6222222222222222, 'test_acc': 0.53925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.6555555555555556, 'test_acc': 0.5268}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.6944444444444444, 'test_acc': 0.5239}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.6833333333333333, 'test_acc': 0.5649}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.7277777777777777, 'test_acc': 0.55345}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.6111111111111112, 'test_acc': 0.5267}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.7555555555555555, 'test_acc': 0.54805}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.6833333333333333, 'test_acc': 0.58345}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.6222222222222222, 'test_acc': 0.52085}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.6611111111111111, 'test_acc': 0.51805}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.6277777777777778, 'test_acc': 0.5409}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.65, 'test_acc': 0.5478}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.65, 'test_acc': 0.5199}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.6666666666666666, 'test_acc': 0.52565}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.7, 'test_acc': 0.54535}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.7222222222222222, 'test_acc': 0.5383}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 20, 'train_acc': 0.7055555555555556, 'test_acc': 0.56585}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 21, 'train_acc': 0.6666666666666666, 'test_acc': 0.55955}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 22, 'train_acc': 0.55, 'test_acc': 0.52705}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 23, 'train_acc': 0.6333333333333333, 'test_acc': 0.5115}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 3, 'repeat': 24, 'train_acc': 0.7277777777777777, 'test_acc': 0.58515}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.7722222222222223, 'test_acc': 0.5877}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8333333333333334, 'test_acc': 0.6139}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.8, 'test_acc': 0.61145}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8, 'test_acc': 0.583}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.7722222222222223, 'test_acc': 0.6011}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.7722222222222223, 'test_acc': 0.6162}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8333333333333334, 'test_acc': 0.5893}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8055555555555556, 'test_acc': 0.6174}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.7777777777777778, 'test_acc': 0.5796}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.7666666666666667, 'test_acc': 0.59995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.8833333333333333, 'test_acc': 0.5911}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8611111111111112, 'test_acc': 0.6233}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8, 'test_acc': 0.5954}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.8222222222222222, 'test_acc': 0.6138}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.7666666666666667, 'test_acc': 0.6086}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8166666666666667, 'test_acc': 0.60725}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.7777777777777778, 'test_acc': 0.58775}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.7833333333333333, 'test_acc': 0.60025}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.85, 'test_acc': 0.589}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8277777777777777, 'test_acc': 0.5829}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 20, 'train_acc': 0.7777777777777778, 'test_acc': 0.58385}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 21, 'train_acc': 0.7222222222222222, 'test_acc': 0.58925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 22, 'train_acc': 0.7722222222222223, 'test_acc': 0.57205}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 23, 'train_acc': 0.7944444444444444, 'test_acc': 0.5927}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 4, 'repeat': 24, 'train_acc': 0.8222222222222222, 'test_acc': 0.60935}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9222222222222223, 'test_acc': 0.6319}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8833333333333333, 'test_acc': 0.6365}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.85, 'test_acc': 0.62995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9277777777777778, 'test_acc': 0.62355}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8888888888888888, 'test_acc': 0.62695}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.8944444444444445, 'test_acc': 0.64795}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.8888888888888888, 'test_acc': 0.616}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9111111111111111, 'test_acc': 0.66255}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.8944444444444445, 'test_acc': 0.62815}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.8833333333333333, 'test_acc': 0.6288}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9166666666666666, 'test_acc': 0.61995}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9111111111111111, 'test_acc': 0.64225}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8722222222222222, 'test_acc': 0.6127}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.9222222222222223, 'test_acc': 0.6433}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.9222222222222223, 'test_acc': 0.6365}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.9111111111111111, 'test_acc': 0.6417}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.9222222222222223, 'test_acc': 0.65255}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8777777777777778, 'test_acc': 0.62915}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9111111111111111, 'test_acc': 0.6284}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9444444444444444, 'test_acc': 0.61325}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 20, 'train_acc': 0.9, 'test_acc': 0.6087}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 21, 'train_acc': 0.8888888888888888, 'test_acc': 0.6294}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 22, 'train_acc': 0.9055555555555556, 'test_acc': 0.617}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 23, 'train_acc': 0.8888888888888888, 'test_acc': 0.64405}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 5, 'repeat': 24, 'train_acc': 0.9166666666666666, 'test_acc': 0.63875}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8611111111111112, 'test_acc': 0.59905}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8777777777777778, 'test_acc': 0.62075}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8388888888888889, 'test_acc': 0.59665}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.9, 'test_acc': 0.59595}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8333333333333334, 'test_acc': 0.6023}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.8833333333333333, 'test_acc': 0.6123}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.8722222222222222, 'test_acc': 0.5928}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.8722222222222222, 'test_acc': 0.6223}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8222222222222222, 'test_acc': 0.5921}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.8555555555555555, 'test_acc': 0.5989}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8166666666666667, 'test_acc': 0.5943}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.9055555555555556, 'test_acc': 0.6188}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.85, 'test_acc': 0.6065}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.8444444444444444, 'test_acc': 0.6171}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8833333333333333, 'test_acc': 0.5784}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.8666666666666667, 'test_acc': 0.5931}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.8944444444444445, 'test_acc': 0.61635}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8833333333333333, 'test_acc': 0.5961}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.9166666666666666, 'test_acc': 0.60025}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8944444444444445, 'test_acc': 0.57515}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 20, 'train_acc': 0.85, 'test_acc': 0.59925}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 21, 'train_acc': 0.8333333333333334, 'test_acc': 0.59855}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 22, 'train_acc': 0.8555555555555555, 'test_acc': 0.58825}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 23, 'train_acc': 0.8777777777777778, 'test_acc': 0.5991}, {'lr': 0.001, 'sampled_percentage': 0.01, 'sampled_cut_point': 6, 'repeat': 24, 'train_acc': 0.8722222222222222, 'test_acc': 0.5882}]
finetune3 = [{'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.6955555555555556, 'test_acc': 0.6504}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.7155555555555555, 'test_acc': 0.63905}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.7144444444444444, 'test_acc': 0.6419}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.6983333333333334, 'test_acc': 0.64965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.7027777777777777, 'test_acc': 0.63285}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 5, 'train_acc': 0.7105555555555556, 'test_acc': 0.6449}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 6, 'train_acc': 0.6955555555555556, 'test_acc': 0.63585}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 7, 'train_acc': 0.7022222222222222, 'test_acc': 0.6487}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 8, 'train_acc': 0.6933333333333334, 'test_acc': 0.6459}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 9, 'train_acc': 0.6861111111111111, 'test_acc': 0.6249}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 10, 'train_acc': 0.6955555555555556, 'test_acc': 0.6449}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 11, 'train_acc': 0.6855555555555556, 'test_acc': 0.6222}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 12, 'train_acc': 0.7166666666666667, 'test_acc': 0.6432}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 13, 'train_acc': 0.7305555555555555, 'test_acc': 0.6614}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 14, 'train_acc': 0.6588888888888889, 'test_acc': 0.62425}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 15, 'train_acc': 0.6822222222222222, 'test_acc': 0.63365}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 16, 'train_acc': 0.6983333333333334, 'test_acc': 0.63965}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 17, 'train_acc': 0.705, 'test_acc': 0.6434}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 18, 'train_acc': 0.6944444444444444, 'test_acc': 0.6118}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 0, 'repeat': 19, 'train_acc': 0.7466666666666667, 'test_acc': 0.66395}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.7805555555555556, 'test_acc': 0.7129}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.7111111111111111, 'test_acc': 0.6482}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.7966666666666666, 'test_acc': 0.72775}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.7666666666666667, 'test_acc': 0.71515}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.7083333333333334, 'test_acc': 0.65555}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 5, 'train_acc': 0.7333333333333333, 'test_acc': 0.6772}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 6, 'train_acc': 0.7877777777777778, 'test_acc': 0.7115}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 7, 'train_acc': 0.745, 'test_acc': 0.68155}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 8, 'train_acc': 0.7861111111111111, 'test_acc': 0.71925}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 9, 'train_acc': 0.6916666666666667, 'test_acc': 0.64195}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 10, 'train_acc': 0.7661111111111111, 'test_acc': 0.7143}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 11, 'train_acc': 0.7983333333333333, 'test_acc': 0.7186}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 12, 'train_acc': 0.7611111111111111, 'test_acc': 0.6976}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 13, 'train_acc': 0.7444444444444445, 'test_acc': 0.6976}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 14, 'train_acc': 0.76, 'test_acc': 0.72085}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 15, 'train_acc': 0.7422222222222222, 'test_acc': 0.69075}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 16, 'train_acc': 0.7622222222222222, 'test_acc': 0.6969}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 17, 'train_acc': 0.7494444444444445, 'test_acc': 0.6924}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 18, 'train_acc': 0.745, 'test_acc': 0.69785}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 1, 'repeat': 19, 'train_acc': 0.7311111111111112, 'test_acc': 0.6795}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.8511111111111112, 'test_acc': 0.7712}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.8377777777777777, 'test_acc': 0.76345}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.8177777777777778, 'test_acc': 0.73975}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.7772222222222223, 'test_acc': 0.7304}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.8155555555555556, 'test_acc': 0.74875}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 5, 'train_acc': 0.8333333333333334, 'test_acc': 0.75535}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 6, 'train_acc': 0.8194444444444444, 'test_acc': 0.75595}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 7, 'train_acc': 0.8172222222222222, 'test_acc': 0.74875}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 8, 'train_acc': 0.8527777777777777, 'test_acc': 0.7682}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 9, 'train_acc': 0.8472222222222222, 'test_acc': 0.77215}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 10, 'train_acc': 0.84, 'test_acc': 0.7609}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 11, 'train_acc': 0.8122222222222222, 'test_acc': 0.7389}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 12, 'train_acc': 0.8427777777777777, 'test_acc': 0.76445}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 13, 'train_acc': 0.8355555555555556, 'test_acc': 0.76125}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 14, 'train_acc': 0.7877777777777778, 'test_acc': 0.7325}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 15, 'train_acc': 0.8238888888888889, 'test_acc': 0.7486}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 16, 'train_acc': 0.7638888888888888, 'test_acc': 0.69575}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 17, 'train_acc': 0.8283333333333334, 'test_acc': 0.75535}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 18, 'train_acc': 0.8344444444444444, 'test_acc': 0.7586}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 2, 'repeat': 19, 'train_acc': 0.8055555555555556, 'test_acc': 0.73215}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.8516666666666667, 'test_acc': 0.75805}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.8505555555555555, 'test_acc': 0.7652}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.8483333333333334, 'test_acc': 0.75995}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.8372222222222222, 'test_acc': 0.76905}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.8411111111111111, 'test_acc': 0.76065}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 5, 'train_acc': 0.8516666666666667, 'test_acc': 0.77035}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 6, 'train_acc': 0.8405555555555555, 'test_acc': 0.7652}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 7, 'train_acc': 0.8722222222222222, 'test_acc': 0.77955}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 8, 'train_acc': 0.8522222222222222, 'test_acc': 0.7605}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 9, 'train_acc': 0.8672222222222222, 'test_acc': 0.7815}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 10, 'train_acc': 0.8533333333333334, 'test_acc': 0.76355}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 11, 'train_acc': 0.8827777777777778, 'test_acc': 0.784}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 12, 'train_acc': 0.8538888888888889, 'test_acc': 0.76515}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 13, 'train_acc': 0.8611111111111112, 'test_acc': 0.7777}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 14, 'train_acc': 0.8516666666666667, 'test_acc': 0.7788}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 15, 'train_acc': 0.8638888888888889, 'test_acc': 0.76205}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 16, 'train_acc': 0.8438888888888889, 'test_acc': 0.7573}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 17, 'train_acc': 0.8572222222222222, 'test_acc': 0.7734}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 18, 'train_acc': 0.8522222222222222, 'test_acc': 0.7699}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 3, 'repeat': 19, 'train_acc': 0.8433333333333334, 'test_acc': 0.7604}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.8855555555555555, 'test_acc': 0.77505}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.8988888888888888, 'test_acc': 0.78125}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.88, 'test_acc': 0.7675}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.8711111111111111, 'test_acc': 0.76455}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.8661111111111112, 'test_acc': 0.7732}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 5, 'train_acc': 0.8811111111111111, 'test_acc': 0.77145}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 6, 'train_acc': 0.8811111111111111, 'test_acc': 0.7811}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 7, 'train_acc': 0.8872222222222222, 'test_acc': 0.7767}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 8, 'train_acc': 0.8861111111111111, 'test_acc': 0.77165}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 9, 'train_acc': 0.8827777777777778, 'test_acc': 0.7828}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 10, 'train_acc': 0.8505555555555555, 'test_acc': 0.74715}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 11, 'train_acc': 0.8955555555555555, 'test_acc': 0.77895}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 12, 'train_acc': 0.8961111111111111, 'test_acc': 0.7742}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 13, 'train_acc': 0.9072222222222223, 'test_acc': 0.7876}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 14, 'train_acc': 0.8672222222222222, 'test_acc': 0.7703}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 15, 'train_acc': 0.8827777777777778, 'test_acc': 0.77525}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 16, 'train_acc': 0.8966666666666666, 'test_acc': 0.78075}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 17, 'train_acc': 0.8922222222222222, 'test_acc': 0.7765}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 18, 'train_acc': 0.8661111111111112, 'test_acc': 0.77075}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 4, 'repeat': 19, 'train_acc': 0.8655555555555555, 'test_acc': 0.76945}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.8938888888888888, 'test_acc': 0.77085}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.8983333333333333, 'test_acc': 0.76725}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9044444444444445, 'test_acc': 0.7677}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.8977777777777778, 'test_acc': 0.7734}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.8877777777777778, 'test_acc': 0.76635}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 5, 'train_acc': 0.9, 'test_acc': 0.76755}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 6, 'train_acc': 0.8572222222222222, 'test_acc': 0.73995}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 7, 'train_acc': 0.9027777777777778, 'test_acc': 0.76565}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 8, 'train_acc': 0.885, 'test_acc': 0.75545}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 9, 'train_acc': 0.9088888888888889, 'test_acc': 0.7766}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 10, 'train_acc': 0.9066666666666666, 'test_acc': 0.7702}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 11, 'train_acc': 0.9038888888888889, 'test_acc': 0.7691}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 12, 'train_acc': 0.8788888888888889, 'test_acc': 0.75305}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 13, 'train_acc': 0.8811111111111111, 'test_acc': 0.75995}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 14, 'train_acc': 0.8911111111111111, 'test_acc': 0.7642}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 15, 'train_acc': 0.8772222222222222, 'test_acc': 0.7595}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 16, 'train_acc': 0.9088888888888889, 'test_acc': 0.7718}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 17, 'train_acc': 0.8988888888888888, 'test_acc': 0.77275}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 18, 'train_acc': 0.9038888888888889, 'test_acc': 0.7707}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 5, 'repeat': 19, 'train_acc': 0.9033333333333333, 'test_acc': 0.7701}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8272222222222222, 'test_acc': 0.73455}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8377777777777777, 'test_acc': 0.7297}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8261111111111111, 'test_acc': 0.72905}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8233333333333334, 'test_acc': 0.7271}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8161111111111111, 'test_acc': 0.7318}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 5, 'train_acc': 0.8233333333333334, 'test_acc': 0.73125}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 6, 'train_acc': 0.8261111111111111, 'test_acc': 0.7275}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 7, 'train_acc': 0.8261111111111111, 'test_acc': 0.7318}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 8, 'train_acc': 0.8433333333333334, 'test_acc': 0.733}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 9, 'train_acc': 0.8338888888888889, 'test_acc': 0.72845}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 10, 'train_acc': 0.8266666666666667, 'test_acc': 0.73325}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 11, 'train_acc': 0.8233333333333334, 'test_acc': 0.72895}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 12, 'train_acc': 0.8233333333333334, 'test_acc': 0.73265}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 13, 'train_acc': 0.8316666666666667, 'test_acc': 0.73455}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 14, 'train_acc': 0.8066666666666666, 'test_acc': 0.7339}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 15, 'train_acc': 0.8083333333333333, 'test_acc': 0.72735}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 16, 'train_acc': 0.8311111111111111, 'test_acc': 0.7328}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 17, 'train_acc': 0.8238888888888889, 'test_acc': 0.7334}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 18, 'train_acc': 0.8233333333333334, 'test_acc': 0.73}, {'lr': 0.001, 'sampled_percentage': 0.1, 'sampled_cut_point': 6, 'repeat': 19, 'train_acc': 0.8438888888888889, 'test_acc': 0.73055}]
finetune4 = [{'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.9225555555555556, 'test_acc': 0.88365}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9215555555555556, 'test_acc': 0.8831}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9128888888888889, 'test_acc': 0.8708}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.903, 'test_acc': 0.8696}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.912, 'test_acc': 0.87415}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9138888888888889, 'test_acc': 0.87695}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9231111111111111, 'test_acc': 0.8859}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9343333333333333, 'test_acc': 0.89425}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9323333333333333, 'test_acc': 0.8952}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9125555555555556, 'test_acc': 0.88255}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9506666666666667, 'test_acc': 0.90625}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9138888888888889, 'test_acc': 0.8809}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.951, 'test_acc': 0.9059}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.944, 'test_acc': 0.90565}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9366666666666666, 'test_acc': 0.8956}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9368888888888889, 'test_acc': 0.89095}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.94, 'test_acc': 0.8963}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9265555555555556, 'test_acc': 0.8819}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9271111111111111, 'test_acc': 0.89}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9435555555555556, 'test_acc': 0.89755}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9354444444444444, 'test_acc': 0.8832}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9394444444444444, 'test_acc': 0.88675}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9421111111111111, 'test_acc': 0.88445}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.938, 'test_acc': 0.88795}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9343333333333333, 'test_acc': 0.88175}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9227777777777778, 'test_acc': 0.86045}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9251111111111111, 'test_acc': 0.8645}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.9078888888888889, 'test_acc': 0.8503}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9252222222222222, 'test_acc': 0.86805}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9161111111111111, 'test_acc': 0.8587}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8195555555555556, 'test_acc': 0.7849}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8147777777777778, 'test_acc': 0.78555}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8152222222222222, 'test_acc': 0.78235}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8072222222222222, 'test_acc': 0.78385}, {'lr': 0.001, 'sampled_percentage': 0.5, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8144444444444444, 'test_acc': 0.78475}]
finetune5 = [{'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 0, 'train_acc': 0.950824237023836, 'test_acc': 0.9441}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 1, 'train_acc': 0.9482624192470483, 'test_acc': 0.94185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 2, 'train_acc': 0.9488193361550457, 'test_acc': 0.9431}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 3, 'train_acc': 0.9488193361550457, 'test_acc': 0.9428}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 0, 'repeat': 4, 'train_acc': 0.9573401648474048, 'test_acc': 0.95045}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 0, 'train_acc': 0.9389062151926932, 'test_acc': 0.93385}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 1, 'train_acc': 0.9473156605034528, 'test_acc': 0.94105}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 2, 'train_acc': 0.9478168857206505, 'test_acc': 0.94195}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 3, 'train_acc': 0.9553352639786143, 'test_acc': 0.9488}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 1, 'repeat': 4, 'train_acc': 0.9581755402094008, 'test_acc': 0.95185}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 0, 'train_acc': 0.9665849855201604, 'test_acc': 0.9593}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 1, 'train_acc': 0.9464245934506572, 'test_acc': 0.9397}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 2, 'train_acc': 0.9457562931610604, 'test_acc': 0.94015}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 3, 'train_acc': 0.959567832479394, 'test_acc': 0.95295}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 2, 'repeat': 4, 'train_acc': 0.9538872800178213, 'test_acc': 0.947}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 0, 'train_acc': 0.9622410336377812, 'test_acc': 0.9535}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 1, 'train_acc': 0.9599576743149922, 'test_acc': 0.953}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 2, 'train_acc': 0.9511026954778347, 'test_acc': 0.94275}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 3, 'train_acc': 0.9636333259077745, 'test_acc': 0.9561}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 3, 'repeat': 4, 'train_acc': 0.9567275562486077, 'test_acc': 0.9495}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 0, 'train_acc': 0.9583983069725996, 'test_acc': 0.9486}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 1, 'train_acc': 0.9555580307418133, 'test_acc': 0.94625}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 2, 'train_acc': 0.9487079527734462, 'test_acc': 0.93915}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 3, 'train_acc': 0.9441412341278681, 'test_acc': 0.93525}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 4, 'repeat': 4, 'train_acc': 0.9482067275562486, 'test_acc': 0.9394}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 0, 'train_acc': 0.9263755847627534, 'test_acc': 0.91685}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 1, 'train_acc': 0.9332256627311205, 'test_acc': 0.9229}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 2, 'train_acc': 0.934172421474716, 'test_acc': 0.9233}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 3, 'train_acc': 0.9263755847627534, 'test_acc': 0.9166}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 5, 'repeat': 4, 'train_acc': 0.9297170862107373, 'test_acc': 0.92045}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 0, 'train_acc': 0.8095901091557139, 'test_acc': 0.80515}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 1, 'train_acc': 0.8073067498329249, 'test_acc': 0.803}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 2, 'train_acc': 0.8002895967921586, 'test_acc': 0.7964}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 3, 'train_acc': 0.8090331922477166, 'test_acc': 0.80505}, {'lr': 0.001, 'sampled_percentage': 1.0, 'sampled_cut_point': 6, 'repeat': 4, 'train_acc': 0.8070282913789263, 'test_acc': 0.80285}]


In [6]:
results = finetune1 + finetune2 + finetune3 + finetune4 + finetune5

In [10]:
# save fine-tuning results
params_tmp = copy.deepcopy(params)
del params_tmp["device"]
params_tmp["activation_function"] = str(params_tmp["activation_function"])
results = [params_tmp] + results

with open(f'results/cifarstyle_results_cifar_reversed.json', 'w') as f:
    json.dump(results, f)
results = results[1:]